In [ ]:
import pandas as pd

df = pd.read_csv("./DBs/RAC_par_mois.csv", sep=',')

In [12]:
import pandas as pd

df_1 = pd.read_csv("./utils/Population_2024.csv", sep=',')
df_melted = pd.melt(
    df_1,
    id_vars=['Tranche_age'],
    value_vars=['Hommes', 'Femmes'],
    var_name='Sexe',
    value_name='Population'
)
df_melted.head()


,Tranche_age,Sexe,Population
0,0-19,Hommes,8075000
1,20-29,Hommes,3967000
2,30-39,Hommes,4105000
3,40-49,Hommes,4173000
4,50-59,Hommes,4391000


In [13]:
tranche_age_mapping = {0:'0-19',
                       20:'20-29',
                       30:'30-39',
                       40:'40-49',
                       50:'50-59',
                       60:'60-69',
                       70:'70-79',
                       80:'80+'}

sexe_mapping = {1:'Hommes',
                2:'Femmes'}

In [ ]:
year_of_interest = 2024

df_filtered = df[(df['AGE_BEN_SNDS']!=99) & (df['Year'] == year_of_interest) & (df['BEN_SEX_COD'] != 0)]
                 
df_filtered['AGE_BEN_SNDS'] = df_filtered['AGE_BEN_SNDS'].apply(lambda x: tranche_age_mapping.get(x, x))

df_filtered['BEN_SEX_COD'] = df_filtered['BEN_SEX_COD'].apply(lambda x: sexe_mapping.get(x, x))




In [17]:
grouped_df = df_filtered.drop(columns = ['Year', 'Month']).groupby(['AGE_BEN_SNDS', 'BEN_SEX_COD']).sum()
grouped_df.reset_index(inplace=True)
grouped_df.columns = ['Tranche_age', 'Sexe', 'RAC', 'montant_payé']
grouped_df.head()

,Tranche_age,Sexe,RAC,montant_payé
0,0-19,Femmes,3.239398e+09,7.562893e+09
1,0-19,Hommes,3.146707e+09,8.080126e+09
2,20-29,Femmes,1.868235e+09,5.390379e+09
3,20-29,Hommes,1.207528e+09,3.528516e+09
4,30-39,Femmes,2.481996e+09,7.741492e+09


In [18]:
merged_df = pd.merge(df_melted, grouped_df, how='outer', on=['Tranche_age', 'Sexe'])
merged_df['montant_payé_par_personne'] = merged_df['montant_payé'] / merged_df['Population']
merged_df['RAC_par_personne'] = merged_df['RAC'] / merged_df['Population']
meaned_df = merged_df.drop(columns = ['Tranche_age']).groupby(['Sexe']).mean().reset_index()
meaned_df['Tranche_age'] = 'Moyenne'

final_df = pd.concat([merged_df, meaned_df], ignore_index=True)
final_df.to_csv('./Dépense et RAC par sexe et par age 2024.csv', sep=',', index=False)